In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_md
!pip install sentence_transformers
!pip install pyvis
!pip install keybert

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.6)
2021-06-18 19:04:47.816792: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
  Using cached https://files.pythonhosted.org/packages/07/d1/e87844ec86e96df7364f21af2263ad6030c0d727660ae89935c7af56a540/pyvis-0.1.9-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bb/1a/f2db026d4d682303793559f1c2bb425ba3ec0d6fd7ac63397790443f2461/jsonpickle-2.0.0-py2.py3-none-any.whl
  Created wheel for keybert: filename=keybert-0.3.0-cp37-none-any.whl size=19436 sha256=5ba3ce30d31c6e164f0911c1cc04fea3a48079c04a150803a36689a09e40a1ff
  Stored in directory: /root/.cache/pip/wheels/0c/ab/2b/bc129895ee956866333f364f1b3beb20efa788e074c03e6b34
Successfully built keybert


In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Sample corpus
documents = ["Insure parcel ,Clarify shipping method ,Write package label ,Request three offers ,select offer from logistic company,package goods,prepare picking up of goods,Ship goods,Special handling?,Insurance necessary?,shipment prepared,Logistics,Secretary,Warehouse",
             "Package goods ,Determine shipping type ,Check if shipment is required ,Write package label ,Get quote company A ,Get quote company B ,Get quote company C ,Select shipping  ,company ,Take parcel insurance ,Pickup goods ,Customer order ,Shipment required ,no insurance ,special shipping ,normal post ,no shipment required ,insurance required ,Warehousemen ,Secretary ,Logistics department head ,Logistics company"]

In [4]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)
documents_df=pd.DataFrame(documents,columns=['documents'])

In [5]:
# removing special characters and stop words from the text
stop_words_l=stopwords.words('english')
documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l and len(w) > 1) )

In [6]:
documents_df

,documents,documents_cleaned
0,"Insure parcel ,Clarify shipping method ,Write package label ,Request three offers ,select offer from logistic company,package goods,prepare picking up of goods,Ship goods,Special handling?,Insurance necessary?,shipment prepared,Logistics,Secretary,Warehouse",insure parcel clarify shipping method write package label request three offers select offer logistic company package goods prepare picking goods ship goods special handling insurance necessary shipment prepared logistics secretary warehouse
1,"Package goods ,Determine shipping type ,Check if shipment is required ,Write package label ,Get quote company A ,Get quote company B ,Get quote company C ,Select shipping ,company ,Take parcel insurance ,Pickup goods ,Customer order ,Shipment required ,no insurance ,special shipping ,normal post ,no shipment required ,insurance required ,Warehousemen ,Secretary ,Logistics department head ,Logistics company",package goods determine shipping type check shipment required write package label get quote company get quote company get quote company select shipping company take parcel insurance pickup goods customer order shipment required no insurance special shipping normal post no shipment required insurance required warehousemen secretary logistics department head logistics company


In [7]:
bert_model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [8]:
import spacy
import en_core_web_md
nlp = en_core_web_md.load()

In [9]:
new_concepts = documents_df.documents[0].split(',')
existing_concepts = documents_df.documents[1].split(',')

In [10]:
def extract_noun_chunks(concepts):
  spacy_lst = []
  for element in concepts:
      element = " ".join([w.lower() for w in element.split()])
      doc = nlp(element)
      if len(list(doc.noun_chunks)) >0 :
          for chunk in doc.noun_chunks:
              #print(element,"--",chunk.text) if len(chunk.text.split()) > 1 else print("POSSIBLE ERROR!",element,"--",chunk.text)
              spacy_lst.append(chunk.text)
      else:
          #print("ERROR!",pos_tag(word_tokenize(element)))
          spacy_lst.append(element)
  return list(set(spacy_lst))

In [11]:
new_concept_chunks = extract_noun_chunks(new_concepts)  
new_concept_chunks

['three offers',
 'goods',
 'special handling',
 'warehouse',
 'shipping method',
 'insurance',
 'package goods',
 'select offer',
 'parcel',
 'logistics',
 'logistic company',
 'ship goods',
 'package label',
 'secretary',
 'shipment']

In [12]:
existing_concept_chunks = extract_noun_chunks(existing_concepts)
existing_concept_chunks

['warehousemen',
 'package label',
 'secretary',
 'special shipping',
 'select shipping',
 'quote company',
 'package goods',
 'customer order',
 'no insurance',
 'logistics company',
 'company',
 'parcel insurance',
 'b',
 'insurance',
 'shipping type',
 'logistics department head',
 'no shipment',
 'normal post',
 'pickup goods',
 'shipment']

In [13]:
#Get rid of concepts that contains another concepts in them and detect relationships.
def len_func(e):
  return len(e.split())
def substr_op(concepts):
  lst_substr = []
  dict_related = {}
  lst_extr = []
  concepts.sort(key=len_func, reverse=False)
  for idx in range(len(concepts)):
    element = concepts[idx]
    for idy in range(len(concepts)):  
      element_2 = concepts[idy]
      if idx != idy:
        if len(element.split()) > 1:
          if len(set(element.split()).intersection(set(element_2.split()))) > 1:
            #print(idx,idy,element, element_2.split())
            if element not in lst_substr:
                lst_substr.append(element_2)
                dict_related.setdefault(element,[]).append(element_2)
                #print("Related to",element, "-->", element_2) 
        else:
          if len(set(element.split()).intersection(set(element_2.split()))) > 0:
            #print("#Related to",element, "-->", element_2)
            dict_related.setdefault(element,[]).append(element_2)
            lst_extr.append(element_2)
  for i in lst_extr:
    lst_substr.append(i)
  for k in dict_related.keys():
    lst_substr.remove(k) if k in lst_substr else 1
  return list(set(concepts) - set(lst_substr)), dict_related

In [14]:
result = substr_op(existing_concept_chunks)
existing_concept_chunks = result[0]

print(existing_concept_chunks)
result[1]

['warehousemen', 'company', 'select shipping', 'insurance', 'b', 'shipping type', 'package goods', 'logistics department head', 'customer order', 'normal post', 'pickup goods', 'package label', 'secretary', 'special shipping', 'shipment']


{'company': ['quote company', 'logistics company'],
 'insurance': ['no insurance', 'parcel insurance'],
 'shipment': ['no shipment']}

In [15]:
result_2  =substr_op(new_concept_chunks)
new_concept_chunks = result_2[0]

print(new_concept_chunks)
result_2[1]

['goods', 'three offers', 'warehouse', 'special handling', 'insurance', 'shipping method', 'select offer', 'logistics', 'parcel', 'logistic company', 'package label', 'secretary', 'shipment']


{'goods': ['package goods', 'ship goods']}

In [16]:
#Exact string match and not an element in dictionary keys
matching_concepts = [el for el in list(set(existing_concept_chunks).intersection(set(new_concept_chunks))) if el not in result_2[1].keys() and el not in result[1].keys()]
matching_concepts

['package label', 'secretary']

In [17]:
#Exact string match but contains noun in dictionary keys for further investigation.
matching_but_different_concepts = [el for el in list(set(existing_concept_chunks).intersection(set(new_concept_chunks))) if el in result_2[1].keys() or el in result[1].keys()]
matching_but_different_concepts

['shipment', 'insurance']

In [18]:
#Nouns that are different from new model UNION nouns with props.
exact_missing_concepts = list((set(existing_concept_chunks) - set(new_concept_chunks)).union(set(result[1].keys())))
exact_missing_concepts

['warehousemen',
 'company',
 'select shipping',
 'b',
 'logistics department head',
 'shipping type',
 'package goods',
 'insurance',
 'customer order',
 'normal post',
 'pickup goods',
 'special shipping',
 'shipment']

In [19]:
#Nouns that are different from existing models UNION nouns with props.
new_misaligned_concepts = list((set(new_concept_chunks) - set(existing_concept_chunks)).union(set(result_2[1].keys())))
new_misaligned_concepts

['special handling',
 'three offers',
 'goods',
 'warehouse',
 'shipping method',
 'select offer',
 'logistics',
 'parcel',
 'logistic company']

In [20]:
#Candidates
candidate_concepts = list(set(exact_missing_concepts).union(set(new_misaligned_concepts)))
candidate_concepts

['warehousemen',
 'company',
 'special handling',
 'goods',
 'shipping method',
 'select offer',
 'parcel',
 'special shipping',
 'three offers',
 'select shipping',
 'warehouse',
 'logistics department head',
 'b',
 'shipping type',
 'package goods',
 'insurance',
 'logistics',
 'customer order',
 'logistic company',
 'normal post',
 'pickup goods',
 'shipment']

In [21]:
props = {}
covered_concepts = []
different_concepts = []
for c in candidate_concepts:
  if c in list(result[1].keys()) and c in list(result_2[1].keys()):
    if len(set(result[1].get(c)).union(set(result_2[1].get(c))) - set(result[1].get(c)).intersection(set(result_2[1].get(c)))) != 0 :
      props.setdefault(c,[]).extend(list(set(result[1].get(c)).intersection(set(result_2[1].get(c)))))
    covered_concepts.append(c)
  else:
    different_concepts.append(c)
    props.setdefault(c,[]).extend(list(set(result[1].get(c,[])).union(set(result_2[1].get(c,[]))) - set(result[1].get(c,[])).intersection(set(result_2[1].get(c,[])))))

b4 = different_concepts
different_concepts, dict_props = substr_op(different_concepts)
after = different_concepts

for k,v in dict_props.items():
  tmp = list(set(props.get(k,[])+ v))
  props[k] = tmp

for k in list(set(b4)-set(after)):
  """for v in props.values():
    if k in v:
      v.remove(k)"""
  props.pop(k)

In [22]:
#Concepts that are fully covered.
covered_concepts

[]

In [23]:
different_concepts

['warehousemen',
 'goods',
 'company',
 'special handling',
 'shipping method',
 'select offer',
 'parcel',
 'special shipping',
 'three offers',
 'select shipping',
 'warehouse',
 'b',
 'insurance',
 'shipping type',
 'logistics',
 'customer order',
 'normal post',
 'shipment']

In [24]:
#Concepts that have covered with differences.
matching_but_different_concepts = [x for x in matching_but_different_concepts if x not in covered_concepts]
matching_but_different_concepts

['shipment', 'insurance']

In [25]:
#Concepts that are exist in domain but not in new model.
unseen_concepts = [x for x in different_concepts if x not in matching_but_different_concepts]
unseen_concepts

['warehousemen',
 'goods',
 'company',
 'special handling',
 'shipping method',
 'select offer',
 'parcel',
 'special shipping',
 'three offers',
 'select shipping',
 'warehouse',
 'b',
 'shipping type',
 'logistics',
 'customer order',
 'normal post']

In [26]:
props

{'b': [],
 'company': ['quote company', 'logistic company', 'logistics company'],
 'customer order': [],
 'goods': ['package goods', 'pickup goods', 'ship goods'],
 'insurance': ['no insurance', 'parcel insurance'],
 'logistics': ['logistics department head'],
 'normal post': [],
 'parcel': [],
 'select offer': [],
 'select shipping': [],
 'shipment': ['no shipment'],
 'shipping method': [],
 'shipping type': [],
 'special handling': [],
 'special shipping': [],
 'three offers': [],
 'warehouse': [],
 'warehousemen': []}

In [27]:
document_embeddings = bert_model.encode(list(props.keys()))
pairwise_similarities=cosine_similarity(document_embeddings)

In [28]:
props_new = {}
threshold= 0.4

kw_model = KeyBERT('paraphrase-mpnet-base-v2')
for word in [w for w in unseen_concepts if len(props.get(w,[])) == 0]:
  idx = list(props.keys()).index(word)
  sim = -1
  cls = ""
  for i in range(pairwise_similarities.shape[1]):
    if i != idx:
      if sim < pairwise_similarities[idx,i] and pairwise_similarities[idx,i] > threshold:
        sim = pairwise_similarities[idx,i]
        cls = list(props.keys())[i]
      #print("Similarity between",list(props.keys())[idx],"and",list(props.keys())[i],"-->",pairwise_similarities[idx,i]) 
  #print("Similarity between",word,"and",cls,"-->",sim) 
  if len(word.split()) >1 and len(cls.split())>1:
    doc = " ".join([word,cls])
    keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,1))
    if len(keywords) > 0:
      cls = keywords[0][0]

  if word in props_new.keys() and cls in props_new.values():    
    props_new.setdefault(word,[]).append(cls)
  elif cls =="":
    props_new.setdefault(word,[]).append(cls) 
  else:
    props_new.setdefault(cls,[]).append(word) 

In [29]:
props.update(props_new)
for k in list(props.keys()):
    if not props[k]:
        del props[k]
props

{'b': [''],
 'company': ['quote company', 'logistic company', 'logistics company'],
 'goods': ['package goods', 'pickup goods', 'ship goods'],
 'insurance': ['no insurance', 'parcel insurance'],
 'logistics': ['logistics department head'],
 'normal post': [''],
 'offers': ['three offers'],
 'shipment': ['parcel'],
 'shipping': ['special handling',
  'shipping method',
  'select offer',
  'special shipping',
  'select shipping',
  'shipping type',
  'customer order'],
 'warehouse': ['warehousemen'],
 'warehousemen': ['warehouse']}

In [30]:
import networkx as nx
d = []
elements =  [props.items()]
colors = ["blue", "red"] #red not in existing models, blues are present in existing models
for i in range(len(elements)):
  for k,v in elements[i]:
    if len(v[0]) > 0:
      for el in v:
        d.append({"source": k, "target" : el,"type":"Undirected", "weight" : 1, "color": colors[i]})
    else:
        d.append({"source": k, "target" : k,"type":"Undirected", "weight" : 1, "color": "red"})
df = pd.DataFrame(d)
G = nx.from_pandas_edgelist(df,source="source", target = "target", edge_attr = True)

In [33]:
df

,source,target,type,weight,color
0,warehousemen,warehouse,Undirected,1,blue
1,company,quote company,Undirected,1,blue
2,company,logistic company,Undirected,1,blue
3,company,logistics company,Undirected,1,blue
4,goods,package goods,Undirected,1,blue
5,goods,pickup goods,Undirected,1,blue
6,goods,ship goods,Undirected,1,blue
7,warehouse,warehousemen,Undirected,1,blue
8,b,b,Undirected,1,red
9,insurance,no insurance,Undirected,1,blue


In [32]:
import pyvis
import IPython
from pyvis.network import Network

net = Network(notebook=True)
net.from_nx(G)
net.save_graph("graph.html")
IPython.display.HTML(filename="graph.html")